<a href="https://colab.research.google.com/github/marianefn/Project_Lattices/blob/main/Project_Lattices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Θέμα 8

## Βρες αν το διάνυσμα (43, 69, 95, 110) ανήκει στο πλέγμα

In [1]:
import numpy as np

# Ορισμός των πινάκων A και του διανύσματος v
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

B = np.array([[5, 8, 11, 13],
              [4, 6, 8, 9],
              [8, 9, 10, 10],
              [4, 6, 9, 10]])

v = np.array([43, 69, 95, 110])

# Λύση για το σύστημα εξισώσεων A * x = v (γραμμικός συνδυασμός των γραμμών του A)
coefficients_A, resids, rank, s = np.linalg.lstsq(A.T, v, rcond=None)

# Έλεγχος αν το διάνυσμα ανήκει στο πλέγμα του A (αν τα υπόλοιπα είναι πολύ μικρά)
print("Συντελεστές για τις γραμμές του πίνακα A:", coefficients_A)
print("Υπόλοιπα για A:", resids)

# Λύση για το σύστημα εξισώσεων B * x = v (γραμμικός συνδυασμός των γραμμών του B)
coefficients_B, resids_b, rank_b, s_b = np.linalg.lstsq(B.T, v, rcond=None)

# Έλεγχος αν το διάνυσμα ανήκει στο πλέγμα του B (αν τα υπόλοιπα είναι πολύ μικρά)
print("\nΣυντελεστές για τις γραμμές του πίνακα B:", coefficients_B)
print("Υπόλοιπα για B:", resids_b)


Συντελεστές για τις γραμμές του πίνακα A: [14. 11. 10. 11.]
Υπόλοιπα για A: []

Συντελεστές για τις γραμμές του πίνακα B: [ 3.00000000e+00  9.00000000e+00 -1.00000000e+00 -4.17443857e-14]
Υπόλοιπα για B: []


## Βρες ορίζουσα πίνακα Α

In [2]:
import numpy as np

# Ορισμός του πίνακα A (ή B)
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

# Υπολογισμός του όγκου του πλέγματος (ορίζουσα του πίνακα A)
volume_A = np.abs(np.linalg.det(A))

print("Ο όγκος του πλέγματος που παράγεται από τις γραμμές του πίνακα A είναι:", volume_A)

Ο όγκος του πλέγματος που παράγεται από τις γραμμές του πίνακα A είναι: 5.000000000000001


## Σημεία πλέγματος μέσα σε κύκλο και τετράγωνο

In [3]:
import numpy as np

# Ορισμός του πίνακα A
A = np.array([[1, 2, 3, 4],
              [-1, 1, 2, 3],
              [4, 3, 2, 1],
              [0, 0, 1, 1]])

B = np.array([[5, 8, 11, 13],
              [4, 6, 8, 9],
              [8, 9, 10, 10],
              [4, 6, 9, 10]])


# Συνάρτηση για να ελέγξουμε αν ένα διάνυσμα ανήκει στο πλέγμα
def is_in_lattice(vector, matrix):
    try:
        # Λύνουμε το σύστημα A * c = vector
        coefficients, resids, rank, s = np.linalg.lstsq(matrix.T, vector, rcond=None)
        # Ελέγχουμε αν τα υπόλοιπα (resids) είναι κοντά στο μηδέν
        if not np.allclose(np.dot(matrix.T, coefficients), vector):
            return False

        # Ελέγχουμε αν όλοι οι συντελεστές είναι ακέραιοι
        return np.allclose(coefficients, np.round(coefficients))
    except:
        return False

# Συνάρτηση για να υπολογίσουμε τα σημεία εντός του κύκλου
def points_in_circle(radius, matrix):
    count = 0
    for x in range(-radius, radius + 1):
        for y in range(-radius, radius + 1):
            if x**2 + y**2 < radius**2:  # Εντός του κύκλου
                vector = np.array([x, y, 0, 0])  # Μορφή (x, y, 0, 0)
                if is_in_lattice(vector, matrix):
                    count += 1
    return count

# Συνάρτηση για να υπολογίσουμε τα σημεία εντός του τετραγώνου
def points_in_square(side_length, matrix):
    count = 0
    half_side = side_length // 2
    for x in range(-half_side, half_side + 1):
        for y in range(-half_side, half_side + 1):
          if abs(x + y) + abs(x - y) < side_length:
            vector = np.array([x, y, 0, 0])  # Μορφή (x, y, 0, 0)
            if is_in_lattice(vector, matrix):
                count += 1
    return count

# Ρυθμίσεις
radius = 10  # Ακτίνα του κύκλου
side_length = 10  # Πλευρά του τετραγώνου

# Υπολογισμός
points_circle = points_in_circle(radius, B)
points_square = points_in_square(side_length, B)

print(f"Σημεία του πλέγματος μέσα στον κύκλο με ακτίνα 10: {points_circle}")
print(f"Σημεία του πλέγματος μέσα στο τετράγωνο με πλευρά 10: {points_square}")


Σημεία του πλέγματος μέσα στον κύκλο με ακτίνα 10: 61
Σημεία του πλέγματος μέσα στο τετράγωνο με πλευρά 10: 17


# Θέμα 3

In [4]:
import numpy as np

def are_linearly_independent(vec1, vec2):
    """
    Ελέγχει αν δύο διανύσματα είναι γραμμικώς ανεξάρτητα.

    :param vec1: Το πρώτο διάνυσμα (λίστα ή numpy array)
    :param vec2: Το δεύτερο διάνυσμα (λίστα ή numpy array)
    :return: True αν είναι γραμμικώς ανεξάρτητα, False αν όχι

    ~~ function created with chatgpt
    """
    # Μετατροπή σε numpy arrays για ευκολία
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Έλεγχος αν τα διανύσματα έχουν ίδιο μέγεθος
    if vec1.shape != vec2.shape:
        raise ValueError("Τα διανύσματα πρέπει να έχουν το ίδιο μέγεθος.")

    # Δημιουργία πίνακα από τα δύο διανύσματα
    matrix = np.column_stack((vec1, vec2))

    # Υπολογισμός ορίζουσας
    determinant = np.linalg.det(matrix)

    # Τα διανύσματα είναι γραμμικώς ανεξάρτητα αν η ορίζουσα δεν είναι 0
    return not np.isclose(determinant, 0)

# Παράδειγμα χρήσης
vec1 = [1, 1]
vec2 = [3, 4]

print(are_linearly_independent(vec1, vec2))


True


In [5]:
import numpy as np

def calc_norm_squared(vec):
  """Calculates the Euclidean norm of a vector squared up.
  :param vec: A list or numpy array representing the vector.
  :return: The Euclidean norm of the vector squared up.
  """
  vector = np.array(vec)
  return np.sum(vector**2)

In [6]:
# prompt: create function to calculate the norm of a vector (gemini)

def calc_norm(vec):
  """Calculates the Euclidean norm of a vector.
  :param vec: A list or numpy array representing the vector.
  :return: The Euclidean norm of the vector.
  """
  vector = np.array(vec)
  return np.linalg.norm(vector)

In [7]:
# prompt: create function to swap vectors represented by numpy use an example to swap two vectors

import numpy as np

def swap_vectors(vec1, vec2):
    """Swaps two vectors represented by NumPy arrays.
    :param vec1: The first vector (NumPy array).
    :param vec2: The second vector (NumPy array).
    :return: A tuple containing the swapped vectors (vec2, vec1).
    """
    return vec2, vec1

In [8]:
import numpy as np

def angle_between_vectors(vec1, vec2):
    """ Calculates the angle between two vectors in Z^2.
    :param v1: Το πρώτο διάνυσμα (λίστα ή numpy array)
    :param v2: Το δεύτερο διάνυσμα (λίστα ή numpy array)
    :return: Η γωνία σε μοίρες
    """
    # Transform into numpy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Calculation of dot product
    dot_product = np.dot(vec1, vec2)

    # Calculation of norms of the two vectors
    norm_v1 = np.linalg.norm(vec1)
    norm_v2 = np.linalg.norm(vec2)

    # Calculation of cos(θ)
    cos_theta = dot_product / (norm_v1 * norm_v2)

    # Check that cos(θ) in in [-1, 1] to avoid numeric errors
    cos_theta = np.clip(cos_theta, -1, 1)

    # Calculation in radians
    theta_radians = np.arccos(cos_theta)

    # Transformation in degrees
    theta_degrees = np.degrees(theta_radians)

    return theta_degrees

# Παράδειγμα χρήσης
v1 = [1, 2]
v2 = [3, 4]

angle = angle_between_vectors(v1, v2)
print(f"Η γωνία μεταξύ των διανυσμάτων είναι: {angle:.2f} μοίρες")


Η γωνία μεταξύ των διανυσμάτων είναι: 10.30 μοίρες


In [9]:
# prompt: create function that generates a vector in Z^2 without limitations

import numpy as np

def generate_vector_z2():
    """Generates a random vector in Z^2."""
    x = np.random.randint(-100, 101)
    y = np.random.randint(-100, 101)
    return np.array([x, y])

# Example usage
vector = generate_vector_z2()
vector

array([67, 65])

In [10]:
def create_vector_pairs():
  """ Finds two random linear independent vectors in Z^2
  :return: vec1, vec2"""

  found = False
  vec1 = generate_vector_z2()
  while not found:
    vec2 = generate_vector_z2()
    if are_linearly_independent(vec1,vec2):
      found = True

  return vec1, vec2

vec1, vec2 = create_vector_pairs()
print(vec1,vec2)

[-40   0] [ -25 -100]


## Αλγόριθμος των Gauss-Lagrange

In [11]:
import math


def gauss_lagrange(vec1, vec2):
  """Outputs a Lagrange-Gauss reduced basis for the lattice L.
    :param vec1: The first vector (NumPy array).
    :param vec2: The second vector (NumPy array).
    :return: Basis (b1, b2) for L such that ||b_i|| = λ_i
    """

  b1 = calc_norm_squared(vec1)
  m = np.dot(vec1, vec2)/b1
  vec2 = vec2 - math.ceil(m-0.5)*vec1
  b2 = calc_norm_squared(vec2)

  while calc_norm(vec2) < calc_norm(vec1):
      vec1, vec2 = swap_vectors(vec1, vec2)
      b1 = b2
      m = np.dot(vec1, vec2)/b1
      vec2 = vec2 - math.ceil(m-0.5)*vec1
      b2 = calc_norm_squared(vec2)

  return vec1, vec2


# Requesting the vectors as input
while True:
    try:
        vec1_str = input("Enter the first vector (e.g., 1,2,3): ")
        vec1 = np.array([int(x) for x in vec1_str.split(',')])
        vec2_str = input("Enter the second vector (e.g., 4,5,6): ")
        vec2 = np.array([int(x) for x in vec2_str.split(',')])

        if not are_linearly_independent(vec1,vec2):
            print("Vectors must have the same dimension. Please enter again.")
            continue

        break
    except ValueError:
        print("Invalid input. Please enter numbers separated by commas.")

# Checking linear independence
if are_linearly_independent(vec1, vec2):
    print("The vectors are linearly independent.")
else:
    print("The vectors are linearly dependent.")

GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)
print("Gauss-Lagrange b1 =",GL_vec1)
print("Gauss-Lagrange b2 =",GL_vec2)

l1 = calc_norm(GL_vec1)
print("l1 =",l1)



Enter the first vector (e.g., 1,2,3): 1,1
Enter the second vector (e.g., 4,5,6): 3,4
The vectors are linearly independent.
Gauss-Lagrange b1 = [0 1]
Gauss-Lagrange b2 = [1 0]
l1 = 1.0


In [12]:
angle_list = []

for i in range(50):
  # find two linearly independent vectors in Z^2
  vec1, vec2 = create_vector_pairs()
  # calculate the Gauss-Lagrange vectors
  GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)

  # Calulate the angle between the two vectors
  angle = angle_between_vectors(GL_vec1, GL_vec2)
  angle_list.append(angle)


angle_array = np.array(angle_list)
print(angle_array)
print(angle_array.mean())



[ 73.2805452   84.99747011  92.33730586 106.46001481  70.0240707
  91.60140036 118.14886275  92.15391453  90.90758     82.01988625
  77.71689456  70.71365954  88.54436669  90.8867627   75.42803405
  88.04968358  92.69780563  93.52479233  96.72826073 109.37413977
  91.96831749  80.3927384   96.27729849  89.13701786  82.03928601
  66.76794588  94.86994172  99.40313237  86.61965601  96.3967194
  88.78112476  88.19844903  84.40208636 112.12633481  91.14994333
  77.85983797 100.18090647  96.54885527  70.98351646  88.0054307
  90.98754122 103.42688421  85.29385683 106.85839877  85.48601154
  85.79240819  94.19737545  78.31386952  90.          97.09935131]
89.70319371989066


In [13]:
# prompt: find average of numpy array

import numpy as np

angle_array = np.array(angle_list)
print(angle_array.mean())

89.70319371989066


# Θέμα 4

## Αλγόριθμος Gram-Schmidt

In [14]:
import numpy as np

def gram_schmidt(basis):
    """
    Gram-Schmidt Algorithm to create an orthogonal basis.
    :param basis: a list of vecotrs (NumPy arrays) which create the basis B.
    :return orthogonal_basis: list of orthogonal vectors
    :return mu: Gram-Schmidt matrix (numpy array) with μ_ij cofacotrs

    ~ used chatgpt as a reference based on Alrgorithm 24 from Chapter 17
    Lattice Basis Reduction  of the book “Mathematics of Public Key Cryptography” by Steven Galbraithv
    """
    n = len(basis)
    # Initialize the orthogonal basis and the Gram-Schmidt matrix
    b_star = np.zeros_like(basis, dtype=float)
    mu = np.zeros((n, n), dtype=float)

    for i in range(n):
        b_star[i] = basis[i]
        for j in range(i):
            mu[i, j] = np.dot(basis[i], b_star[j]) / np.dot(b_star[j], b_star[j])
            b_star[i] -= mu[i, j] * b_star[j]

    return b_star, mu

# Παράδειγμα χρήσης
basis = [
    np.array([1.0, 1.0]),
    np.array([1.0, 0.0])
]

orthogonal_basis, mu = gram_schmidt(basis)

# Εκτύπωση αποτελεσμάτων
print("Ορθογώνια βάση:")
for vec in orthogonal_basis:
    print(vec)

print("\nGram-Schmidt πίνακας (μ):")
print(mu)


Ορθογώνια βάση:
[1. 1.]
[ 0.5 -0.5]

Gram-Schmidt πίνακας (μ):
[[0.  0. ]
 [0.5 0. ]]


## Αλγόριθμος LLL

In [15]:
import math
import numpy as np

def lll_algorithm(basis, delta):
  """
    LLL algorithm with Euclidean norm
    :param orthogonal_basis: a list of vecotrs (NumPy arrays) which the basis B.
    :param delta: typically choose δ = 3/4
    :return basis: LLL reduced basis b1, . . . , bn (numpy array)
  """
  n = len(basis)
  orthogonal_basis, mu = gram_schmidt(basis)
  # Compute Bi = <{b_i}*,{b_i}*> = ||{b_i}*||^2
  B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])

  k=1
  while k < n:
    # Perform size reduction
    for j in range(k - 1, -1, -1):
      # q = math.floor(mu[k,j]+0.5)
      q = round(mu[k, j])
      basis[k] -= q*basis[j]
      mu[k, :k + 1] -= q * mu[j, :k + 1]

    orthogonal_basis, mu = gram_schmidt(basis)
    B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])

    # Check Lov´asz condition
    if B[k] < (delta-mu[k,k-1]**2)*B[k-1]:
        # Swap basis[k] and basis[k-1]
        basis[k], basis[k-1] = basis[k-1], basis[k].copy()
        orthogonal_basis, mu = gram_schmidt(basis)
        B = np.array([np.dot(orthogonal_basis[i], orthogonal_basis[i]) for i in range(n)])
        k = max(1, k - 1)
    else:
      k+=1

  return basis



# Examples
delta = 0.75
basis = np.array([[1, 1, 1], [1, 0, 2], [0, 1, 1]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[10,1], [1, 9]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[1, -1, 3], [1, 0, 5], [1, 2, 6]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[-36,32], [-32, -27]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[-1, -1, -2, -4, -1], [-2, -2, 4, -4, 2], [4, 2, -1, 1, 4], [1, -1, 1, -1, -1],[-1, 1, -1, 1, -30]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)

basis = np.array([[42, -36, -5], [-13, 9, 30]])
lll_reduced_basis = lll_algorithm(basis,delta)
print("LLL Reduced Basis:")
print(lll_reduced_basis)


LLL Reduced Basis:
[[-1  0  0]
 [ 0 -1  1]
 [ 0  1  1]]
LLL Reduced Basis:
[[10  1]
 [ 1  9]]
LLL Reduced Basis:
[[ 1 -1  0]
 [-1  0  1]
 [ 1  1  1]]
LLL Reduced Basis:
[[-36  32]
 [-32 -27]]
LLL Reduced Basis:
[[ 1 -1  1 -1 -1]
 [-2  0 -3 -3  0]
 [-3 -1  3 -3  3]
 [ 5  1  0  0  3]
 [ 3  9  1 -1 -2]]
LLL Reduced Basis:
[[-13   9  30]
 [ 29 -27  25]]


## πρώτο διάνυσμα που δίνει ο LLL έχει μεγαλύτερο μήκος από το δεύτερο διάνυσμα

In [16]:
def generate_random_lattice(dim, seed=None):
    """
    Create a ranom dimxdim lattice
    :parm dim: dimention of the lattice.
    :parm seed:optional seed (default None).
    :returns a numpy array representing the lattice.
    ~ created with chatgpt
    """
    if seed is not None:
        np.random.seed(seed)
    return np.random.randint(-10, 10, size=(dim, dim))

def check_lll_norm_condition(max_iterations=100):
  """
  Checks random lattices (up to max_iterations) until it finds one where the
  norm of the first vector of the LLL-reduced lattice is greater than the second.
  :parm max_iterations: The maximum number of iterations to perform.
  :returns a tuple containing the original base, the LLL-reduced basis and the norms if the condition is met,
    None otherwise.

  ~ used Gemini as reference for naming
  """

  for _ in range(max_iterations):
    basis = generate_random_lattice(4)

    # Apply the LLL algorithm
    lll_basis = lll_algorithm(basis.copy(), delta=0.75)

    # Check the condition
    norm_first = calc_norm(lll_basis[0])
    norm_second = calc_norm(lll_basis[1])

    if norm_first > norm_second:
        return  basis, lll_basis, (norm_first, norm_second)

  return None


result = check_lll_norm_condition()

if result:
    basis, lll_basis, norms = result
    print("Original basis found satisfying the condition:")
    print(basis)
    print("LLL-reduced basis found satisfying the condition:")
    print(lll_basis)
    print("Norms:", norms)
else:
    print("No LLL-reduced basis found satisfying the condition within the given iterations.")

Original basis found satisfying the condition:
[[ -3   0  -8   6]
 [ -6  -7   5   5]
 [  5  -5   3  -4]
 [  2 -10   7   7]]
LLL-reduced basis found satisfying the condition:
[[ 2 -5 -5  2]
 [ 3  2 -1  6]
 [-8  3 -2 -2]
 [-6 -7  5  5]]
Norms: (7.615773105863909, 7.0710678118654755)


## Τυχαία εκτέλεση LLL και Gauss-Lagrange σε πλέγματα βαθμίδας 2

In [17]:
def random_executions(iterations):
  """
  Checks <iterations> random lattices
  :parm iterations: The number of iterations to perform.
  """
  counter = 0
  for _ in range(iterations):
    # Generate two random linearly independent vectors in Z^2
    vec1, vec2 = create_vector_pairs()
    basis = np.array([vec1, vec2])

    # Apply the LLL algorithm
    lll_basis = lll_algorithm(basis.copy(), delta=0.75)

    # Apply the Gauss-Lagrance algorithm
    GL_vec1, GL_vec2 = gauss_lagrange(vec1, vec2)
    gauss_basis = np.array([GL_vec1, GL_vec2])

    """
    print("Original basis:")
    print(basis)
    print("LLL-reduced basis:")
    print(lll_basis)
    print("Gauss-Lagrange-reduced basis:")
    print(gauss_basis)
    """
    if  np.array_equal(lll_basis, gauss_basis):
      #print("The reduced bases are the same")
      counter += 1
    #else:
      #print("The reduced bases are NOT the same")
    #print("----------------------------------------")
  print(counter, "out of ",iterations, "exapmles were the same")

  return None


run = random_executions(100)

89 out of  100 exapmles were the same


# Θέμα 10
## Αλγόριθμος του Bambai

In [18]:
def bambai_algorithm(basis, t):
  """
  Babai’s nearest plane algorithm

  :param basis: Basis matrix (n x m) (NumPy arrays) describing the lattice.
  :param t: Target vector (NumPy arrays), m-dimensional
  :return x: The closest lattice point to t.
  """
  n = len(basis)
  # Apply the LLL algorithm
  lll_basis = lll_algorithm(basis.copy(), delta=0.75)

  # Apply Gram-Schmidt Algorithm to create an orthogonal basis
  orthogonal_basis, mu = gram_schmidt(lll_basis.copy())

  b = t.copy()
  for j in range(n - 1, -1, -1):
    c_j = round(np.dot(b,orthogonal_basis[j])/np.dot(orthogonal_basis[j],orthogonal_basis[j]))
    b -= c_j* lll_basis[j]
  return t-b


# Examples
#B = np.array([[1, 0], [0, 1]])
#t = np.array([2.7, 3.4])
#closest_point = bambai_algorithm(B, t)
#print("The closest lattice point is:", closest_point)

B = np.array([[137, 312], [215, -187]])
t = np.array([53172, 81743])
closest_point = bambai_algorithm(B, t)
print("The closest lattice point is:", closest_point)


The closest lattice point is: [53159 81818]


# Θέμα 11
## Αλγόριθμος KFP

In [ ]:
def kfp_enumeration(basis, R):
    """
    KFP enumeration algorithm returns all vectors of L which are smaller than R

    :param basis: Basis matrix (n x n) (NumPy arrays) describing the lattice.
    :param R: possitive number constraint for the norm
    :return:  A set of lattice vectors satisfying ||x|| <= R
    """
    n = len(basis)
    # compute {μ_{ij}}
    b_star, mu = gram_schmidt(basis.copy())
    # Compute Bi = ||{b_i}*||^2
    B = np.array([np.dot(b_star[i], b_star[i]) for i in range(n)])

    x = np.zeros(n, dtype=int)      # Initialize x vector
    c = np.zeros(n, dtype=float)    # Initialize c vector
    l = np.zeros(n, dtype=float)    # Initialize l vector
    sumli = 0
    S = set()                       # Set to store lattice vectors
    i = 0

    while i < n:
        # Update c[i] as the summation of μ[j, i] * x[j] for j=i+1 to n
        c[i] = -sum(x[j] * mu[j][i] for j in range(i+1, n))

        l[i] = B[i] * ((x[i] - c[i])**2)
        sumli = sum(l[i:n])

        if sumli <= R**2:
            if i == 0:
                S.add(tuple(sum(x[j] * basis[j] for j in range(n))))
                x[0] += 1
            else:
                i -= 1
                summation = sum(mu[j, i] * x[j] for j in range(i + 1, n))
                # xi ← left part of the interval Ii
                x[i] = math.ceil(-summation - np.sqrt((R**2 - sum(l[i+1:n])) / B[i]))
        else:
            i += 1
            if i < n:
              x[i] += 1
    return S

# Example
basis = np.array([[3, 6, 13], [11, 3, 15], [12, 12, 0]])
R = np.sqrt(214)

result = kfp_enumeration(basis, R)
print("Lattice vectors satisfying ||x|| <= R:")
for vector in result:
    print(vector)


Lattice vectors satisfying ||x|| <= R:
(8, -3, 2)
(0, 0, 0)
(3, 6, 13)


In [ ]:
# Παράδειγμα χρήσης
basis = np.array([[-1, -1, -2, -4, -1], [-2, -2, 4, -4, 2], [4, 2, -1, 1, 4], [1, -1, 1, -1, -1], [-1, 1, -1, 1, 30]])
R = np.sqrt(40)

result = kfp_enumeration(basis, R)
print("Vectors satisfying the condition:")
for vector in result:
    print(vector)

Vectors satisfying the condition:
(5, 1, 0, 0, 3)
(3, 1, -3, -3, 3)
(1, -1, 1, -1, -1)
(0, -2, -1, -5, -2)
(2, 0, 3, 3, 0)
(3, -1, 4, 2, -1)
(-1, -1, -2, -4, -1)
(4, 2, -1, 1, 4)
(0, 0, 0, 0, 0)
(4, 0, -2, 2, -4)
(2, -2, 2, -2, -2)
(4, 0, -2, -4, 2)
(3, 1, -3, 3, -3)


# Θέμα 12
## ́Αλγόριθμος Schnorr-Euchner

In [ ]:
def schnorr_euchner_algorithm(basis, R):
  """
  KFP enumeration algorithm returns all vectors of L which are smaller than R

    :param basis: Basis matrix (n x n) (NumPy arrays) describing the lattice.
    :param R: possitive number constraint for the norm
    :return:  A set of lattice vectors satisfying ||x|| <= R
  """
  n = len(basis)
  # compute {μ_{ij}}
  b_star, mu = gram_schmidt(basis.copy())
  # Compute Bi = ||{b_i}*||^2
  B = np.array([np.dot(b_star[i], b_star[i]) for i in range(n)])

  x = np.zeros(n, dtype=int)      # Initialize x vector
  c = np.zeros(n, dtype=float)    # Initialize c vector
  l = np.zeros(n, dtype=float)    # Initialize l vector

  # Δx ← (1, 0n−1)
  delta_x = np.zeros(n)
  delta_x[0] = 1

  # Δ2x ← (1, (−1)n−1)
  delta_2x = np.ones(n)
  delta_2x[1:] = -1

  sumli = 0
  S = set()                       # Set to store lattice vectors
  i = 0

  while i < n:
    c[i] = -sum(x[j]*mu[j][i] for j in range(i+1, n))
    l[i] = B[i]*((x[i]-c[i])**2)
    sumli = sum(l[i:n])
    if sumli <= R**2 and i == 0:
      S.add(tuple(sum(x[j]*basis[j] for j in range(n))))
    if sumli <= R**2 and i > 0:
      i -= 1
      c[i] = -sum(x[j]*mu[j][i] for j in range(i+1, n)) # center of the interval In+1−i
      x[i] = round(c[i])
      delta_x[i] = 0
      if c[i] < x[i]:
        delta_2x[i] = 1
      else:
        delta_2x[i] = -1
    elif i ==n:         # i.e. if sumli > R2 and i = n
      break
    else:               # i.e. if sumli > R2 or i = 1
      i +=1
      if i < n:
        delta_2x[i] = -delta_2x[i]
        delta_x[i] = -delta_x[i] + delta_2x[i]
        x[i] = x[i] + delta_x[i]
  return S


# Example
basis = np.array([[3, 6, 13], [11, 3, 15], [12, 12, 0]])
R = np.sqrt(214)

result = schnorr_euchner_algorithm(basis, R)
print("Lattice vectors satisfying ||x|| <= R:")
for vector in result:
    print(vector)


Lattice vectors satisfying ||x|| <= R:
(8, -3, 2)
(0, 0, 0)
(-8, 3, -2)


In [ ]:
# Παράδειγμα χρήσης
basis = np.array([[-1, -1, -2, -4, -1], [-2, -2, 4, -4, 2], [4, 2, -1, 1, 4], [1, -1, 1, -1, -1], [-1, 1, -1, 1, 30]])
R = np.sqrt(40)

result = schnorr_euchner_algorithm(basis, R)
print("Vectors satisfying the condition:")
for vector in result:
    print(vector)

Vectors satisfying the condition:
(-1, 1, -1, 1, 1)
(-3, -1, 3, 3, -3)
(5, 1, 0, 0, 3)
(1, -1, 1, -1, -1)
(3, 1, -3, -3, 3)
(-2, 2, -2, 2, 2)
(-4, 0, 2, -2, 4)
(-5, -1, 0, 0, -3)
(-3, -1, 3, -3, 3)
(0, 0, 0, 0, 0)
(4, 0, -2, 2, -4)
(2, -2, 2, -2, -2)
(3, 1, -3, 3, -3)


## enumeration with prunning και prunning vector

In [25]:
import math

def compute_pruning_vector(n):
    """
    Computes the pruning vector a_i = min(1, sqrt(1.05^(i/n))).
        :param n (int): Dimension of the lattice.
        :return: list of float: Pruning vector.
    """
    return [min(1, math.sqrt(1.05**(i/n))) for i in range(n)]

def schnorr_euchner_algorithm_with_prunning(basis, R):
  """
  KFP enumeration algorithm returns all vectors of L which are smaller than R

    :param basis: Basis matrix (n x n) (NumPy arrays) describing the lattice.
    :param R: possitive number constraint for the norm
    :return:  A set of lattice vectors satisfying ||x|| <= R
  """
  n = len(basis)
  # compute {μ_{ij}}
  b_star, mu = gram_schmidt(basis.copy())
  # Compute Bi = ||{b_i}*||^2
  B = np.array([np.dot(b_star[i], b_star[i]) for i in range(n)])

  x = np.zeros(n, dtype=int)      # Initialize x vector
  c = np.zeros(n)                 # Initialize c vector
  l = np.zeros(n)                 # Initialize l vector

  # Δx ← (1, 0n−1)
  delta_x = np.zeros(n)
  delta_x[0] = 1

  # Δ2x ← (1, (−1)n−1)
  delta_2x = np.ones(n)
  delta_2x[1:] = -1

  sumli = 0
  S = set()                       # Set to store lattice vectors
  i = 0

  # compute prunning vvector
  a = compute_pruning_vector(n)

  while i < n:
    c[i] = -sum(x[j]*mu[j][i] for j in range(i+1, n))
    l[i] = B[i]*((x[i]-c[i])**2)
    sumli = sum(l[i:n])
    if sumli <= (a[n-1-i]*R)**2 and i == 0:
      S.add(tuple(sum(x[j]*basis[j] for j in range(n))))
    if sumli <= (a[n-1-i]*R)**2 and i > 0:
      i -= 1
      c[i] = -sum(x[j]*mu[j][i] for j in range(i+1, n)) # center of the interval In+1−i
      x[i] = round(c[i])
      delta_x[i] = 0
      if c[i] < x[i]:
        delta_2x[i] = 1
      else:
        delta_2x[i] = -1
    elif i ==n:         # i.e. if sumli > R2 and i = n
      break
    else:               # i.e. if sumli > R2 or i = 1
      i +=1
      if i < n:
        delta_2x[i] = -delta_2x[i]
        delta_x[i] = -delta_x[i] + delta_2x[i]
        x[i] = x[i] + delta_x[i]
  return S


# Example
basis = np.array([[3, 6, 13], [11, 3, 15], [12, 12, 0]])
R = np.sqrt(214)

result = schnorr_euchner_algorithm_with_prunning(basis, R)
print("Lattice vectors satisfying ||x|| <= R:")
for vector in result:
    print(vector)


Lattice vectors satisfying ||x|| <= R:
(8, -3, 2)
(0, 0, 0)
(-8, 3, -2)
